In [23]:
class SimpleClass():
    
    def __init__(self, name):
        print("Hello " + name)
        
    def yell(self):
        print("YELLING")
        

In [8]:
s = "world"

In [9]:
type(s)

str

In [25]:
x = SimpleClass('nome')

Hello nome


In [13]:
x.yell()

YELLING


In [26]:
class ExtendedClass (SimpleClass):
    def __init__(self):
        
        super().__init__('world')
        print("EXTENDED")

In [27]:
y = ExtendedClass()

Hello world
EXTENDED


In [17]:
y.yell()

YELLING


## Operation

In [101]:
class Operation():
    
    def __init__(self, input_nodes=[]):
        
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
        _default_graph.operations.append(self)
    def compute(self):
        pass
    

In [102]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [103]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [104]:
class matmul(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

In [105]:
class Placeholder():
    
    def __init__(self):
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [106]:
class Variable():
    
    def __init__(self, initial_value=None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)
        

In [120]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph
        _default_graph = self
        

In [121]:
g = Graph()

In [123]:
g.set_as_default()

In [124]:
A = Variable(10)

In [125]:
b = Variable(1)

In [126]:
x = Placeholder()

In [127]:
y = multiply(A,x)

In [128]:
z = add(y,b)

In [129]:
import numpy as np

In [130]:
def traverse_postorder(operation):
    
    nodes_postorder = []
    
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    return nodes_postorder


In [131]:
class Session():
    
    def run(self, operation, feed_dict={}):
        
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            
            elif type(node) == Variable:
                node.output = node.value
                
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                
                
                node.output = node.compute(*node.inputs)
            if type(node.output) == list:
                node.output = np.array(node.output)
        return operation.output

In [132]:
sess = Session()

In [133]:
result = sess.run(operation=z, feed_dict={x:10})

In [134]:
result

101

In [135]:
g = Graph()

g.set_as_default()  

A = Variable([[10,20],[30,40]])
b = Variable([1,2])

x = Placeholder()
y = matmul(A,x)

z = add(y,b)

In [136]:
sess = Session()

In [137]:
sess.run(operation=z, feed_dict={x:10})

array([[101, 202],
       [301, 402]])